In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

import nltk 
import random 
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('wordnet') 

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# import csv
# import urllib.request as urllib2

url="https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv"
df = pd.read_csv(url, sep='\t')

df.head(10)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [0]:
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,46557,2265,goods,2
1,63416,3209,Triumph of Love is a very silly movie,1
2,121037,6483,to ever get under the skin,1
3,129504,6963,prostituted,2
4,98133,5141,that stinks so badly of hard-sell image-monger...,0
5,127386,6851,Solaris '',2
6,87401,4533,directed this movie so much as produced it -- ...,1
7,48597,2373,You have enough finely tuned acting to compens...,2
8,134850,7278,Chinese immigrant 's experiences,2
9,121266,6493,"the National Lampoon film franchise , too long...",1


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df ['Phrase'], df ['Sentiment'], test_size=0.3, random_state=2003)
documents=[]
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())
for i in range(len(X_train)):
  documents.append([list(word_tokenize(X_train[i])), Y_train[i]]) 

X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  documents.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

documents[0]

[['It',
  "'s",
  'a',
  'pity',
  'that',
  '-LRB-',
  'Nelson',
  "'s",
  '-RRB-',
  'achievement',
  'does',
  "n't",
  'match',
  'his',
  'ambition'],
 0]

In [0]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"
#parameters to adjust to see the impact on outcome 
remove_stopwords = True
useStemming = False
useLemma = False
removePuncs = True


In [0]:
for l in range(len(documents)):                   #For each review document 
  label = documents[l][1]                         #Save review label 
  tmpReview = []                                  #Placeholder list for new review 
  for w in documents[l][0]:                       #For each word this is review 
    newWord = w                                   #Set newWork to be the updated word 
    if remove_stopwords and (w in stopwords_en):  #if the word is a stopword & we want to remove stopwords 
      continue                                    #skip the word and don’t had it to the normalized review 
    if removePuncs and (w in punctuations):       #if the word is a punc. & we want to remove punctuations 
      continue                                    #skip the word and don’t had it to the normalized review 
    if useStemming:
      #if useStemming is set to True 
      #Keep one stemmer commented out 
      #newWord = porter.stem(newWord) #User porter stemmer 
      newWord = lancaster.stem(newWord) #Use Lancaster stemmer 
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    tmpReview.append(newWord)                     #Add normalized word to the tmp review 
  documents[l] = (tmpReview, label)             #Update the reviews list with clean review 
  documents[l] = (' '.join(tmpReview), label) 

print(documents[0])

("It 's pity -LRB- Nelson 's -RRB- achievement n't match ambition", 0)


In [0]:
df = pd.DataFrame(documents, columns=['text', 'sentiment']) 
df.head()



,text,sentiment
0,It 's pity -LRB- Nelson 's -RRB- achievement n...,0
1,survive,2
2,enjoyable occasionally flawed experiment,3
3,probably interest primarily target audience,2
4,When,2


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],  df['sentiment'], test_size=0.3, random_state=2003)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

vectorizer = TfidfVectorizer(max_features = 2000)#, # ngram_range=(1, 1)) 
X = vectorizer.fit_transform(df["text"]) 
Y = df['sentiment'] 
 
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test


Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras import backend as K

batch_size = 64
num_classes = 5
epochs = 10

X_train.shape

(109242, 2000)

In [0]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

In [0]:

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3,
                 activation='relu',
                 input_shape=(2000,1)))

model.add(Conv1D(64, kernel_size=5, activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=1))

model.add(Conv1D(64, kernel_size=5, activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=1))

model.add(Conv1D(64, kernel_size=5, activation='relu'))

# model.add(Dropout(rate = 0.50))

model.add(Flatten())

model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [0]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=10)
# _, accuracy = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





109242/109242 [==============================] - 58s 531us/step - loss: 1.0525 - acc: 0.5862
Epoch 2/10
109242/109242 [==============================] - 45s 411us/step - loss: 0.9510 - acc: 0.6279
Epoch 3/10
109242/109242 [==============================] - 45s 408us/step - loss: 0.8887 - acc: 0.6510
Epoch 4/10
109242/109242 [==============================] - 45s 408us/step - loss: 0.8388 - acc: 0.6710
Epoch 5/10
109242/109242 [==============================] - 44s 406us/step - loss: 0.7903 - acc: 0.6884
Epoch 6/10
109242/109242 [==============================] - 44s 407us/step - loss: 0.7539 - acc: 0.7007
Epoch 7/10
109242/109242 [==============================] - 44s 407us/step - loss: 0.7245 - acc: 0.7118
Epoch 8/10
109242/109242 [==============================] - 44s 406us/step - loss: 0.7016 - acc: 0.7203
Epoch 9/10
109242/109242 [==============================] - 44s 40

In [0]:
from keras.models import load_model

model.save("1095816_1dconv reg.h5")
model = load_model('1095816_1dconv reg.h5')